In [18]:
import numpy as np
import matplotlib.pyplot as plt
import wandb

import os
import sys
import pickle
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

sys.path.append(os.path.join(sys.path[0], '../src'))
import data
import utils
import model
from model import EncoderRNN
from model import DecoderRNN

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


val_acc,▁
val_acc,0.0


## Get training data

In [ ]:
# Change to directory containing English to Tamil transliteration data.
DATA_DIR = "../data/aksharantar_sampled/tam/"
os.chdir(DATA_DIR)
os.listdir()

Load training data

In [3]:
# dataset_train = data.WordDataset() # By default reads 'tam_train.csv'.
# with open('dataset_train.pkl', 'wb') as file:
#     pickle.dump(dataset_train, file)

with open('dataset_train.pkl', 'rb') as file:
    dataset_train = pickle.load(file)

Load validation data

In [4]:
# dataset_valid = data.WordDataset(file='tam_valid.csv')
# with open('dataset_valid.pkl', 'wb') as file:
#     pickle.dump(dataset_valid, file)

with open('dataset_valid.pkl', 'rb') as file:
    dataset_valid = pickle.load(file)

### Test training and validation.

In [ ]:
encoder = EncoderRNN(cell_type='lstm')
decoder = DecoderRNN(cell_type='lstm')
encoder, decoder, loss_seq = utils.train(dataset_train, encoder, decoder, 0.0001, 10000)

In [ ]:
# Plot loss for first 1000 iteration.
# After 1000 iterations it flattens.
plt.plot(loss_seq[:10000])

Validate

In [ ]:
total_predictions = 0
correct_predictions = 0
for i in tqdm(range(len(dataset_valid))):
    x, y = dataset_valid[i]
    encoder.initialize_hidden()
    y_pred = utils.predict(x, encoder, decoder)
    y_word = dataset_valid.decode_tam_word(y)
    y_pred_word = dataset_valid.decode_tam_word(y_pred)
    total_predictions += 1
    if(y_word == y_pred_word):
        correct_predictions += 1
    # print(y_word, " : ", y_pred_word)

print(correct_predictions / total_predictions)

## Perform WandB sweep

In [13]:
wandb.login() 
project_name = 'CS6910-Assignment-3-Sweep'

In [15]:
def sweep(config):
    encoder = EncoderRNN()
    decoder = DecoderRNN()
    # Set parameters.
    encoder.embedding_size = config.embedding_size
    decoder.embedding_size = config.embedding_size
    # encoder.num_layers = config.num_encoder_layers
    # decoder.num_layers = config.num_decoder_layers
    encoder.hidden_size = config.hidden_size
    decoder.hidden_size = config.hidden_size
    encoder.cell_type = config.cell_type
    decoder.cell_type = config.cell_type
    learning_rate = config.learning_rate

    # Training
    print("Training...")

    encoder, decoder, loss_seq = utils.train(dataset_train, encoder, decoder, learning_rate, 10000)

    # Validation

    print("Validating...")

    total_predictions = 0
    correct_predictions = 0
    for i in tqdm(range(len(dataset_valid))):
        x, y = dataset_valid[i]
        encoder.initialize_hidden()
        y_pred = utils.predict(x, encoder, decoder)
        y_word = dataset_valid.decode_tam_word(y)
        y_pred_word = dataset_valid.decode_tam_word(y_pred)
        total_predictions += 1
        if(y_word == y_pred_word):
            correct_predictions += 1

    return (correct_predictions / total_predictions)

In [16]:
def main():
    wandb.init(project = project_name)
    performance = sweep(wandb.config)
    # wandb.log({'acc': performance[0], 'val_acc': performance[1], 'loss': performance[2], 'val_loss': performance[3]})
    wandb.log({'val_acc': performance})

sweep_configuration = {
    'method': 'bayes',
    'metric': {
        'goal': 'maximize',
        'name': 'val_acc'
    },
    'parameters': {
        'learning_rate': {'values': [0.01, 0.001, 0.0001]},
        'embedding_size': {'values': [16, 32]},
        # 'num_encoder_layers': {'values': [1, 2, 3]},
        # 'num_decoder_layers': {'values': [1, 2, 3]},
        'hidden_size': {'values': [16, 32]},
        'cell_type': {'values': ['gru', 'lstm']},
        # 'bidirectional': {'values': [False, True]},
        # 'dropout': {'values': [0.2, 0.3]}
    }
}

sweep_id = wandb.sweep(sweep = sweep_configuration, project = project_name)
wandb.agent(sweep_id, function = main, count = 90)

Create sweep with ID: t4v1k974
Sweep URL: https://wandb.ai/cs6910-cs22m068/CS6910-Assignment-3-Sweep/sweeps/t4v1k974


wandb: Agent Starting Run: pe29unch with config:
wandb: 	cell_type: lstm
wandb: 	embedding_size: 16
wandb: 	hidden_size: 16
wandb: 	learning_rate: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Training...


  0%|          | 0/10000 [00:00<?, ?it/s]/mnt/d/courses/cs6910_dl/assignment_3/notebooks/../src/model.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.out_act(output)
100%|██████████| 10000/10000 [02:19<00:00, 71.71it/s]


Validating...


...

Training...


  0%|          | 0/10000 [00:00<?, ?it/s]/mnt/d/courses/cs6910_dl/assignment_3/notebooks/../src/model.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.out_act(output)
100%|██████████| 10000/10000 [02:11<00:00, 75.97it/s]


Validating...


 40%|███▉      | 1638/4096 [00:33<00:45, 53.48it/s]wandb: Ctrl + C detected. Stopping sweep.


 58%|█████▊    | 2394/4096 [00:51<00:34, 48.65it/s]